In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import numpy as np
import pickle
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import torch.nn as nn
from transformers import BeitForImageClassification, BeitFeatureExtractor
import torch.nn as nn
from torchvision import models

class HybridModel(nn.Module):
    def __init__(self, num_classes, use_pretrained=True, feature_extract=True):
        super(HybridModel, self).__init__()

        # VGG19
        self.vgg = models.vgg19_bn(pretrained=use_pretrained)
        self.set_parameter_requires_grad(self.vgg, feature_extract)
        num_ftrs_vgg = self.vgg.classifier[6].in_features
        self.vgg.classifier[6] = nn.Linear(num_ftrs_vgg, num_classes)

        # Vision Transformer
        self.vit = models.vit_b_16(pretrained=use_pretrained)
        self.set_parameter_requires_grad(self.vit, feature_extract)
        num_ftrs_vit = self.vit.heads.head.in_features
        self.vit.heads.head = nn.Linear(num_ftrs_vit, num_classes)

        # Combined classifier
        self.classifier = nn.Linear(num_classes * 2, num_classes)  # Output from both models

    def set_parameter_requires_grad(self, model, feature_extracting):
        if feature_extracting:
            for param in model.parameters():
                param.requires_grad = False

    def forward(self, x):
        # Forward pass through VGG19
        vgg_out = self.vgg(x)

        # Forward pass through ViT
        vit_out = self.vit(x)

        # Concatenate outputs from both models
        combined_out = torch.cat((vgg_out, vit_out), dim=1)

        # Classifier
        output = self.classifier(combined_out)
        return output


def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "hybrid_vgg_vit":
        """ Hybrid model of VGG19 and ViT """
        model_ft = HybridModel(num_classes, use_pretrained, feature_extract)
        input_size = 224
        return model_ft, input_size


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("aryanfelix/brats-2019-traintestvalid")

print("Path to dataset files:", path)

100%|██████████| 62.9M/62.9M [00:04<00:00, 14.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1


In [5]:
import os

data_dir = "/root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1"
for root, dirs, files in os.walk(data_dir):
    print(f"Root: {root}")
    print("Directories:", dirs)



Root: /root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1
Directories: ['dataset']
Root: /root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1/dataset
Directories: ['valid', 'train', 'test']
Root: /root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1/dataset/valid
Directories: ['yes', 'no']
Root: /root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1/dataset/valid/yes
Directories: []
Root: /root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1/dataset/valid/no
Directories: []
Root: /root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1/dataset/train
Directories: ['yes', 'no']
Root: /root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1/dataset/train/yes
Directories: []
Root: /root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1/dataset/train/no
Directories: []
Root: /root/.cache/kagg

In [11]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from transformers import BeitForImageClassification, BeitFeatureExtractor

# Define the hybrid model class
class HybridModel(nn.Module):
    def __init__(self, num_classes, use_pretrained=True, feature_extract=True):
        super(HybridModel, self).__init__()

        # VGG19
        self.vgg = models.vgg19_bn(pretrained=use_pretrained)
        self.set_parameter_requires_grad(self.vgg, feature_extract)
        num_ftrs_vgg = self.vgg.classifier[6].in_features
        self.vgg.classifier[6] = nn.Linear(num_ftrs_vgg, num_classes)

        # Vision Transformer (ViT)
        self.vit = models.vit_b_16(pretrained=use_pretrained)
        self.set_parameter_requires_grad(self.vit, feature_extract)
        num_ftrs_vit = self.vit.heads.head.in_features
        self.vit.heads.head = nn.Linear(num_ftrs_vit, num_classes)

        # Combined classifier
        self.classifier = nn.Linear(num_classes * 2, num_classes)  # Concatenate outputs from both models

    def set_parameter_requires_grad(self, model, feature_extracting):
        if feature_extracting:
            for param in model.parameters():
                param.requires_grad = False

    def forward(self, x):
        # Forward pass through VGG19
        vgg_out = self.vgg(x)

        # Forward pass through ViT
        vit_out = self.vit(x)

        # Concatenate outputs from both models
        combined_out = torch.cat((vgg_out, vit_out), dim=1)

        # Classifier
        output = self.classifier(combined_out)
        return output

# Initialize the model
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == "hybrid_vgg_vit":
        model_ft = HybridModel(num_classes, use_pretrained, feature_extract)
        input_size = 224
    return model_ft, input_size

# Custom Dataset class
class BraTSDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        image = Image.open(self.file_paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Function to get file paths and labels
def get_brats_data_paths(data_dir):
    file_paths = []
    labels = []

    # Define mappings for labels
    label_mapping = {'yes': 1, 'no': 0}

    # Collect paths and labels from train, valid, and test folders
    for split in ['train', 'valid']:
        split_dir = os.path.join(data_dir, 'dataset', split)

        for label_name in label_mapping.keys():
            label_dir = os.path.join(split_dir, label_name)
            if os.path.exists(label_dir):
                for file_name in os.listdir(label_dir):
                    file_paths.append(os.path.join(label_dir, file_name))
                    labels.append(label_mapping[label_name])

    return file_paths, labels

# Set data directory path
data_dir = "/root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1"

# Load data paths and labels
file_paths, labels = get_brats_data_paths(data_dir)

# Split into train and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create datasets and data loaders
train_dataset = BraTSDataset(train_paths, train_labels, transform=transform)
val_dataset = BraTSDataset(val_paths, val_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize model
num_classes = 2
model_name = "hybrid_vgg_vit"
feature_extract = True
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Define training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_ft.parameters(), lr=0.001)

# Training loop with model saving
num_epochs = 5
for epoch in range(num_epochs):
    model_ft.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model_ft(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track accuracy and loss
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

# Save the trained teacher model
teacher_model_path = "hybrid_teacher_model.pth"
torch.save(model_ft.state_dict(), teacher_model_path)
print(f"Teacher model saved at {teacher_model_path}")

# Validation loop
model_ft.eval()
with torch.no_grad():
    val_loss = 0.0
    correct = 0
    total = 0

    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model_ft(images)
        loss = criterion(outputs, labels)

        val_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_acc = correct / total

print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")



Epoch [1/5], Loss: 0.3614, Accuracy: 0.8600
Epoch [2/5], Loss: 0.1676, Accuracy: 0.9483
Epoch [3/5], Loss: 0.1144, Accuracy: 0.9654
Epoch [4/5], Loss: 0.0891, Accuracy: 0.9733
Epoch [5/5], Loss: 0.0641, Accuracy: 0.9821
Teacher model saved at hybrid_teacher_model.pth
Validation Loss: 0.0518, Validation Accuracy: 0.9867


In [7]:
!pip install efficientnet_pytorch


  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=c9a1daac82984828da63113461d50d5f1bcc7bdb675efecd299a4eefa1c278af
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [18]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from PIL import Image
from torch.optim import Adam
from transformers import BeitForImageClassification, BeitFeatureExtractor

# Define the hybrid model class (Teacher Model)
class HybridModel(nn.Module):
    def __init__(self, num_classes, use_pretrained=True, feature_extract=True):
        super(HybridModel, self).__init__()

        # VGG19
        self.vgg = models.vgg19_bn(pretrained=use_pretrained)
        self.set_parameter_requires_grad(self.vgg, feature_extract)
        num_ftrs_vgg = self.vgg.classifier[6].in_features
        self.vgg.classifier[6] = nn.Linear(num_ftrs_vgg, num_classes)

        # Vision Transformer (ViT)
        self.vit = models.vit_b_16(pretrained=use_pretrained)
        self.set_parameter_requires_grad(self.vit, feature_extract)
        num_ftrs_vit = self.vit.heads.head.in_features
        self.vit.heads.head = nn.Linear(num_ftrs_vit, num_classes)

        # Combined classifier
        self.classifier = nn.Linear(num_classes * 2, num_classes)  # Concatenate outputs from both models

    def set_parameter_requires_grad(self, model, feature_extracting):
        if feature_extracting:
            for param in model.parameters():
                param.requires_grad = False

    def forward(self, x):
        # Forward pass through VGG19
        vgg_out = self.vgg(x)

        # Forward pass through ViT
        vit_out = self.vit(x)

        # Concatenate outputs from both models
        combined_out = torch.cat((vgg_out, vit_out), dim=1)

        # Classifier
        output = self.classifier(combined_out)
        return output

# Initialize the model
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == "hybrid_vgg_vit":
        model_ft = HybridModel(num_classes, use_pretrained, feature_extract)
        input_size = 224
    return model_ft, input_size

# Define the student model (EfficientNet)
def initialize_student_model(num_classes):
    student_model = models.efficientnet_b0(pretrained=True)
    num_ftrs = student_model.classifier[1].in_features
    student_model.classifier[1] = nn.Linear(num_ftrs, num_classes)  # Adjust for the number of classes
    return student_model

# Custom Dataset class for BraTS
class BraTSDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        image = Image.open(self.file_paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Function to get file paths and labels
def get_brats_data_paths(data_dir):
    file_paths = []
    labels = []

    # Define mappings for labels
    label_mapping = {'yes': 1, 'no': 0}

    # Collect paths and labels from train, valid, and test folders
    for split in ['train', 'valid']:
        split_dir = os.path.join(data_dir, 'dataset', split)

        for label_name in label_mapping.keys():
            label_dir = os.path.join(split_dir, label_name)
            if os.path.exists(label_dir):
                for file_name in os.listdir(label_dir):
                    file_paths.append(os.path.join(label_dir, file_name))
                    labels.append(label_mapping[label_name])

    return file_paths, labels

# Set data directory path
data_dir = "/root/.cache/kagglehub/datasets/aryanfelix/brats-2019-traintestvalid/versions/1"

# Load data paths and labels
file_paths, labels = get_brats_data_paths(data_dir)

# Split into train and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create datasets and data loaders
train_dataset = BraTSDataset(train_paths, train_labels, transform=transform)
val_dataset = BraTSDataset(val_paths, val_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize the teacher model
num_classes = 2
model_name = "hybrid_vgg_vit"
feature_extract = True
teacher_model, _ = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
teacher_model.to(device)

# Load the trained teacher model
teacher_model.load_state_dict(torch.load("hybrid_teacher_model.pth"))
teacher_model.eval()  # Set to evaluation mode

# Initialize the student model (EfficientNet)
student_model = initialize_student_model(num_classes)
student_model.to(device)

# Loss functions
criterion_hard = nn.CrossEntropyLoss()
criterion_soft = nn.KLDivLoss(reduction="batchmean")

# Optimizer for student model
optimizer = Adam(student_model.parameters(), lr=0.001)

# Hyperparameters for knowledge distillation
temperature = 3.0
alpha = 0.7  # Balance between hard and soft loss

# Training loop for knowledge distillation
num_epochs = 5
for epoch in range(num_epochs):
    student_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass through the teacher model (no gradient tracking)
        with torch.no_grad():
            teacher_outputs = teacher_model(images)

        # Forward pass through the student model
        student_outputs = student_model(images)

        # Soft labels from teacher (using temperature)
        soft_labels = nn.functional.softmax(teacher_outputs / temperature, dim=1)

        # Compute the losses
        loss_hard = criterion_hard(student_outputs, labels)
        loss_soft = criterion_soft(
            nn.functional.log_softmax(student_outputs / temperature, dim=1), soft_labels
        )
        loss = alpha * loss_soft + (1 - alpha) * loss_hard

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track accuracy and loss
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(student_outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

# Save the student model after training
student_model_path = "efficientnet_student_model.pth"
torch.save(student_model.state_dict(), student_model_path)
print(f"Student model saved at {student_model_path}")

# Validation loop for the student model
student_model.eval()
with torch.no_grad():
    val_loss = 0.0
    correct = 0
    total = 0

    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass through the student model
        outputs = student_model(images)
        loss = criterion_hard(outputs, labels)

        val_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_acc = correct / total

print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")


<ipython-input-18-964beb3e5868>:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model.load_state_dict(torch.load("hybrid_teacher_model.pth"))
/usr/local/lib/pytho

Epoch [1/5], Loss: 0.0803, Accuracy: 0.9533
Epoch [2/5], Loss: 0.0322, Accuracy: 0.9929
Epoch [3/5], Loss: 0.0273, Accuracy: 0.9938
Epoch [4/5], Loss: 0.0293, Accuracy: 0.9950
Epoch [5/5], Loss: 0.0247, Accuracy: 0.9933
Student model saved at efficientnet_student_model.pth
Validation Loss: 0.0370, Validation Accuracy: 0.9917
